In [1]:
!nvidia-smi

Sat Apr 17 09:59:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Imports

In [3]:
!pip install pytorch-lightning

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from enum import Enum

import numpy as np
import csv
import sys
import os
import random
import matplotlib.pyplot as plt
import matplotlib
import time
from math import floor
import tqdm

import pytorch_lightning as pl

# Hyper-parameters

In [5]:
data_dir = '/gdrive/MyDrive/BTP/Data/baseline_data/S08'
test_dir = '/gdrive/MyDrive/BTP/Data/baseline_data/S09'

BATCH_SIZE = 32
n_worker = 2

save_dir = '/gdrive/MyDrive/BTP/Models'
os.makedirs(save_dir,exist_ok=True)

color_space = 1

thresholdBelowMax = 6

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Utilities

#### top k acc

In [6]:
def top_k_acc(y_true:torch.Tensor,y_pred:torch.Tensor,k=1):
    
    y_pred_tpk = torch.topk(y_pred,k,dim=1)[1]
    
    ovr = 0
    pos = 0

    
    for i in range(0,len(y_pred_tpk)):
        if(y_true[i] in y_pred_tpk[i]):
            pos+=1
        ovr+=1
    
    acc = pos/ovr
    
    return acc

In [7]:
def top_k_acc_tie(y_true:torch.Tensor,y_pred:torch.Tensor,k=1):
    pos=0
    ovr=0
    y_pred = y_pred.cpu().detach().numpy()
    y_true = y_true.cpu().detach().numpy()
    y_pred_idx = np.argsort(y_pred)[:,::-1]
    y_pred_val = np.sort(y_pred)[:,::-1]
    for i in range(0,y_pred.shape[0]):
        fin = list()
        idx = y_pred_idx[i]
        val = y_pred_val[i][k-1]
        for j in range(0,len(y_pred_val[i])):
            if(y_pred_val[i][j]>=val):
                fin.append(idx[j])
        if(y_true[i] in fin):
            pos+=1
        ovr+=1
    
    return pos/ovr


#### extras

In [8]:
def sub2ind(array_shape, rows, cols):
    ind = rows*array_shape[1] + cols
    ind[ind < 0] = -1
    ind[ind >= array_shape[0]*array_shape[1]] = -1
    return ind

In [9]:
def ind2sub(array_shape, ind):
    ind[ind < 0] = -1
    ind[ind >= array_shape[0]*array_shape[1]] = -1
    rows = (ind.astype('int') / array_shape[1])
    cols = ind % array_shape[1]
    return (rows, cols)

# Ranking beam output

Reading beam_output and converting them to logarithm scale; Power (dB) (neg) and setting a threshold value below which rounding all to zero

Overall classes: 8*32 = 256

In [10]:
def beamsLogScale(y:np,thresholdBelowMax):
    y_shape = y.shape
    
    for i in range(0,y_shape[0]):            
        thisOutputs = y[i,:]
        logOut = 20*np.log10(thisOutputs + 1e-30)
        minValue = np.amax(logOut) - thresholdBelowMax
        zeroedValueIndices = logOut < minValue
        thisOutputs[zeroedValueIndices]=0
        thisOutputs = thisOutputs / sum(thisOutputs)
        y[i,:] = thisOutputs
    
    return y

In [11]:
def getBeamOutput(output_file:str):
    
    print("Reading dataset...", output_file)
    output_cache_file = np.load(output_file)
    yMatrix = output_cache_file['output_classification']
    
    yMatrix = np.abs(yMatrix)
    yMatrix /= np.max(yMatrix)
    yMatrixShape = yMatrix.shape
    num_classes = yMatrix.shape[1] * yMatrix.shape[2] #Overall 
    
    y = yMatrix.reshape(yMatrix.shape[0],num_classes)
    y = beamsLogScale(y,thresholdBelowMax)
    
    return y,num_classes

# Creating Dataset and Dataloader


## Loading Data

In [12]:
coordURL = os.path.join(data_dir,'coord_input')
imgURL = os.path.join(data_dir,'image_input')
lidarURL = os.path.join(data_dir,'lidar_input')
beamURL = os.path.join(data_dir,'beam_output')

In [13]:
coordURL_t = os.path.join(test_dir,'coord_input')
imgURL_t = os.path.join(test_dir,'image_input')
lidarURL_t = os.path.join(test_dir,'lidar_input')
beamURL_t = os.path.join(test_dir,'beam_output')

In [14]:
coord_train = np.load(coordURL+'/coord_train.npz')['coordinates']
coord_dev = np.load(coordURL+'/coord_validation.npz')['coordinates']

img_train = np.load(imgURL+'/img_input_train_20.npz')['inputs']
img_dev = np.load(imgURL+'/img_input_validation_20.npz')['inputs']

lidar_train = np.load(lidarURL+'/lidar_train.npz')['input']
lidar_dev = np.load(lidarURL+'/lidar_validation.npz')['input']

In [15]:
coord_test = np.load(coordURL_t+'/coord_test.npz')['coordinates']

img_test = np.load(imgURL_t+'/img_input_test_20.npz')['inputs']

lidar_test = np.load(lidarURL_t+'/lidar_test.npz')['input']

In [16]:
y_train,num_classes = getBeamOutput(beamURL+'/beams_output_train.npz')
y_dev,_ = getBeamOutput(beamURL+'/beams_output_validation.npz')
y_test,_ = getBeamOutput(beamURL_t+'/beams_output_test.npz')
y_train = np.argmax(y_train,axis=1)
y_dev = np.argmax(y_dev,axis=1)
y_test = np.argmax(y_test,axis=1)
print(num_classes)

Reading dataset... /gdrive/MyDrive/BTP/Data/baseline_data/S08/beam_output/beams_output_train.npz
Reading dataset... /gdrive/MyDrive/BTP/Data/baseline_data/S08/beam_output/beams_output_validation.npz
Reading dataset... /gdrive/MyDrive/BTP/Data/baseline_data/S09/beam_output/beams_output_test.npz
256


## Creating Dataset and Dataloader

### Custom Dataset

In [17]:
from PIL import Image
from torchvision import transforms
transform_pipe = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [18]:
class custom_dataset(Dataset):
    def __init__(self,coord,img,lidar,label,transform=None):
        self.coord = coord
        self.img = img
        self.lidar = lidar
        self.label =  label
        self.transform = transform
    
    def __getitem__(self,idx):
        sample = dict()

        sample['coord'] = self.coord[idx]
        sample['lidar'] = self.lidar[idx].reshape((10,20,200))
        sample['label'] = self.label[idx]
        
        m,n,o = self.img[idx].shape
        img_sample =  self.img[idx].reshape(o,m,n)[::-1,:,:].copy()

        a,b,c = (img_sample.shape)
        c = int(c/3)

        img1 = img_sample[:,:,0:c]
        img2 = img_sample[:,:,c:2*c]
        img3 = img_sample[:,:,2*c:3*c]

        sample['img1'] = img1
        sample['img2'] = img2
        sample['img3'] = img3

        return sample
    
    def __len__(self):
        return len(self.label)
 

In [19]:
train_dataset = custom_dataset(coord_train,
                               img_train,
                               lidar_train,
                               y_train)

In [20]:
dev_dataset = custom_dataset(coord_dev,
                             img_dev,
                             lidar_dev,
                             y_dev)

In [21]:
test_dataset = custom_dataset(coord_test,
                             img_test,
                             lidar_test,
                             y_test)

In [22]:
print(len(train_dataset))
print(len(dev_dataset))
print(len(test_dataset))

9234
1960
9638


### Dataloader

In [23]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    shuffle = True
    )

In [24]:
dev_loader = DataLoader(
    dev_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    )

In [25]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    )

# Coord_mlp

## Model Class

In [50]:
class LitModel(pl.LightningModule):
    def __init__(self,learning_rate):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(2,4),
            nn.ReLU(),
            nn.Linear(4,16),
            nn.ReLU(),
            nn.Linear(16,64),
            nn.ReLU(),
            nn.Linear(64,256)
        )
        self.CEloss = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
    
    def training_step(self,batch,batch_idx):
        coord = batch['coord'].float()
        label = batch['label']
        y_hat = self.encoder(coord)
        loss = self.CEloss(y_hat,label)
        self.log('my_loss',loss)
        return {'loss':loss,'pred':y_hat,'label':label}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr= self.learning_rate)
        return optimizer

    def training_epoch_end(self,train_out):
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Train Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))


    def validation_step(self,batch,batch_idx):
        coord = batch['coord'].float()
        label = batch['label']
        yhat = self.encoder(coord)
        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Dev Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

    def test_step(self,batch,batch_idx):
        label = batch['label']
        coord = batch['coord'].float()

        yhat = self.encoder(coord)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def test_epoch_end(self,test_out):
        len_out = len(test_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = test_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = test_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)
        print('Test Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

## Running Trainer

In [51]:
def init_weights_coord(m):
    if type(m) == (nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.000) 

In [52]:
# coord_litmodel = LitModel(learning_rate=1e-4)
# coord_litmodel = coord_litmodel.apply(init_weights_coord)

In [ ]:
# trainer_coord = pl.Trainer(reload_dataloaders_every_epoch = True,
#                      gpus=1,
#                      max_epochs = 1,
#                      num_sanity_val_steps=-1,
#                      auto_lr_find = True
#                      )

In [ ]:
# trainer_coord.tune(coord_litmodel,train_loader,dev_loader)
# trainer_coord.fit(coord_litmodel,train_loader,dev_loader)

In [ ]:
# trainer_coord.test(coord_litmodel,test_loader)

# Model Class: light_image

In [32]:
class light_image(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(color_space,8,(3,3),padding=0)
        self.mpool = nn.MaxPool2d((2,2),stride = (1,1))
        self.conv2a = nn.Conv2d(8,8,(3,3))
        self.conv2b = nn.Conv2d(8,8,(3,3))
        self.relu = nn.ReLU()
        self.drop = torch.nn.Dropout(p=0.25)
        self.flatten = nn.Flatten()
        self.fc3 = nn.Linear(12480,256)

    def forward(self,X1,X2,X3):
        b = int(X1.shape[3]/3)
        
        t1 = self.conv_forward(X1)
        t2 = self.conv_forward(X2)
        t3 = self.conv_forward(X3)
        t = torch.cat([t1,t2,t3],dim=1)
        # t = self.fc1(t)
        # t = self.fc2(t)
        t  = self.fc3(t)

        return t


    def conv_forward(self,X1):
        X1 = self.conv1(X1)
        X1 = self.relu(X1)
        X1 = self.mpool(X1)

        X1 = self.conv2a(X1)
        X1 = self.relu(X1)
        X1 = self.mpool(X1)

        X1 = self.conv2b(X1)
        X1 = self.relu(X1)

        X1 = self.drop(X1)
        X1 = self.flatten(X1)
        
        return X1


### Intializing Model

In [33]:
# def init_weights_img(m):
#     if type(m) == (nn.Linear or nn.Conv2d):
#         torch.nn.init.xavier_uniform_(m.weight)
#         m.bias.data.fill_(0.01)  

In [34]:
# model_img = light_image()
# model_img = model_img.apply(init_weights_img)

# Lidar

In [35]:
class lidar_lit(pl.LightningModule):
    def __init__(self,learning_rate = 1e-3):
        super().__init__()

        self.drop_prob = 0.3
        self.drop = nn.Dropout(self.drop_prob)
        
        self.conv1 = nn.Sequential(nn.Conv2d(10,10,(13,13)),nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(10,30,(11,11),padding = (6,6)),nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(30,25,(9,9),padding = (5,5)),nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(25,20,(9,9),padding = (4,4)),nn.ReLU())
        self.conv5 = nn.Sequential(nn.Conv2d(20,15,(5,5),padding = (4,4)),nn.ReLU())
        self.conv6 = nn.Sequential(nn.Conv2d(15,10,(3,3),padding = (2,2)),nn.ReLU())
        self.conv7 = nn.Sequential(nn.Conv2d(10,1,(1,1),padding = (1,1)),nn.ReLU())

        self.m_pool1 = nn.MaxPool2d((2,1))
        self.m_pool2 = nn.MaxPool2d((1,2))

        self.flatten = nn.Flatten()

        self.fc = nn.Sequential(nn.Linear(1456,1024),
                                nn.Linear(1024,512),
                                nn.Linear(512,256)
                                )

        self.CEloss = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self,X):
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.conv3(X)
        X = self.m_pool1(X)
        X = self.drop(X)
        X = self.conv4(X)
        X = self.m_pool2(X)
        X = self.conv5(X)
        X = self.drop(X)
        X = self.conv6(X)
        X = self.conv7(X)
        X = self.flatten(X)

        out = self.fc(X)

        return out
    
    def training_step(self,batch,batch_idx):
        label = batch['label']
        lidar = batch['lidar'].float()

        yhat = self(lidar)

        loss = self.CEloss(yhat,label)
        self.log('my_loss',loss)

        return {'loss':loss,'pred':yhat,'label':label}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr= self.learning_rate)
        return optimizer

    def training_epoch_end(self,train_out):
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Train Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

    def validation_step(self,batch,batch_idx):
        label = batch['label']
        lidar = batch['lidar'].float()

        yhat = self(lidar)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Dev Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

    def test_step(self,batch,batch_idx):
        label = batch['label']
        lidar = batch['lidar'].float()

        yhat = self(lidar)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def test_epoch_end(self,test_out):
        len_out = len(test_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = test_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = test_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Test Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

## Runnning Trainer

In [36]:
# def init_weights_lid(m):
#     if type(m) == (nn.Linear or nn.Conv2d):
#         torch.nn.init.xavier_uniform_(m.weight)
#         m.bias.data.fill_(0.000)  

In [37]:
# model_lidar = lidar_lit()
# model_lidar = model_lidar.apply(init_weights_lid)

In [38]:
# trainer_lidar = pl.Trainer(reload_dataloaders_every_epoch = True,
#                      gpus=1,
#                      max_epochs = 10,
#                      num_sanity_val_steps=-1,
#                      auto_lr_find = True
#                      )

In [39]:
# trainer_lidar.tune(model_lidar,train_loader,dev_loader)
# trainer_lidar.fit(model_lidar,train_loader,dev_loader)

In [40]:
# trainer_lidar.test(model = model_lidar,test_dataloaders=test_loader)

# Resnet

## Model Class

In [41]:
import torchvision.models as models

res18 = models.resnet18(pretrained = True)

for params in res18.parameters():
    params.requires_grad = False

In [42]:
class res18_lit(pl.LightningModule):
    def __init__(self,learning_rate = 1e-3):
        super().__init__()

        self.activation = {}

        self.resnet = res18
        self.resnet.avgpool.register_forward_hook(self.get_res_activation('out',self.activation))
        self.fc1 = nn.Linear(512*3,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,256)

        self.CEloss = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def get_res_activation(self,name, activation):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    def forward(self,X1,X2,X3):
        res_out = self.resnet(X1)
        t1 = self.activation['out'].reshape((BATCH_SIZE,512))
        res_out = self.resnet(X2)
        t2 = self.activation['out'].reshape((BATCH_SIZE,512))
        res_out = self.resnet(X3)
        t3 = self.activation['out'].reshape((BATCH_SIZE,512))

        t = torch.cat([t1,t2,t3],dim = 1)

        out = self.fc1(t)
        out = self.fc2(out)
        out = self.fc3(out)

        return out
    
    def training_step(self,batch,batch_idx):
        label = batch['label']

        img1 = batch['img1'].float()
        img2 = batch['img2'].float()
        img3 = batch['img3'].float()

        y_hat = self(img1,img2,img3)

        loss = self.CEloss(y_hat,label)
        self.log('my_loss',loss)

        return {'loss':loss,'pred':y_hat,'label':label}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr= self.learning_rate)
        return optimizer

    def training_epoch_end(self,train_out):
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print(f'Train Topk accuracies are :  Top 1: {top1}, Top 5: {top5}, Top 10: {top10},Top 50 {top50}')


    def validation_step(self,batch,batch_idx):
        label = batch['label']

        img1 = batch['img1'].float()
        img2 = batch['img2'].float()
        img3 = batch['img3'].float()

        yhat = self(img1,img2,img3)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print(f'Dev Topk accuracies are :  Top 1: {top1}, Top 5: {top5}, Top 10: {top10},Top 50 {top50}')

## Running Model

In [43]:
# model_res = res18_lit()

In [44]:
# sum(p.numel() for p in model_res.parameters() if p.requires_grad)

In [45]:
# trainer_img = pl.Trainer(reload_dataloaders_every_epoch = True,
#                      gpus=1,
#                      max_epochs = 10,
#                      num_sanity_val_steps=-1,
#                      auto_lr_find = True
#                      )

In [46]:
# trainer_img.tune(model_res,train_loader)
# trainer_img.fit(model_res,train_loader,dev_loader)